In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
ksr = pd.read_csv('ksr_clean.csv')

In [3]:
df = pd.read_excel('train.xlsx')

In [8]:
df.head()

,record_name,record_name_2,ref_code,ref_name,ref_unit
0,DIN -рейка оцинкованная ТН35-7.5 100 см (Chint),"DIN-рейка 35х7,5 мм длиной 1000 мм",27.33.13.130.20.2.08.01-0001-000,"DIN-рейки металлические, оцинкованные, размеры...",100 шт
1,Анкерный элемент ТехноНиколь 8*45мм,Анкерный элемент TN 8x4.5,25.94.11.190.01.7.15.01-1169-000,"Анкер грунтовый забивной самораскрывающийся, о...",шт
2,Анкер забивной М10 DRM 12x40 сталь,Анкер втулочный M10,25.94.11.190.01.7.15.01-0037-000,"Анкер забивной латунный, диаметр внутренней ре...",шт
3,Анкер забивной М10/12x40,Анкер втулочный M10,25.94.11.190.01.7.15.01-0037-000,"Анкер забивной латунный, диаметр внутренней ре...",шт
4,Анкер забивной М8 LAZ латунь,Анкер втулочный M8,25.94.11.190.01.7.15.01-0036-000,"Анкер забивной латунный, диаметр внутренней ре...",шт


In [9]:
# df = df.drop(['target', 'id'], axis=1)
df = df[['record_name', 'ref_name']]

In [10]:
df.head()

,record_name,ref_name
0,DIN -рейка оцинкованная ТН35-7.5 100 см (Chint),"DIN-рейки металлические, оцинкованные, размеры..."
1,Анкерный элемент ТехноНиколь 8*45мм,"Анкер грунтовый забивной самораскрывающийся, о..."
2,Анкер забивной М10 DRM 12x40 сталь,"Анкер забивной латунный, диаметр внутренней ре..."
3,Анкер забивной М10/12x40,"Анкер забивной латунный, диаметр внутренней ре..."
4,Анкер забивной М8 LAZ латунь,"Анкер забивной латунный, диаметр внутренней ре..."


In [4]:
from rapidfuzz import fuzz


In [6]:
def calculate_similarity(ksr, pred):
    similarity = []

    for ksr_item in ksr:
        similarity += [fuzz.ratio(ksr_item, pred)]
    return similarity

In [59]:
def top_3(data, i, n_similar):
    data['similarity_score'] = calculate_skill_similarity(data['record_name'], data['record_name'][i])
    data = data.sort_values(by = 'similarity_score')
    return data.iloc[-(n_similar+1):-1]

In [60]:
df['record_name'][7]

'Анкерный стержень 15,0мм без покрытия 1,00м'

In [64]:
top_3(df, 7, 3)

,record_name,ref_name,similarity_score
1,Анкерный элемент ТехноНиколь 8*45мм,"Анкер грунтовый забивной самораскрывающийся, о...",46.153846
9,"Анкерный стержень 15,0мм без покрытия, неоцинк...","Анкер металлический канатный, диаметр грузонес...",82.352941
8,"Анкерный стержень 15,0мм без покрытия, неоцинк...","Анкер металлический канатный, диаметр грузонес...",82.352941


In [50]:
df.shape[0]

2407

In [65]:
def new_df(data, n_similar):
    promt_df = pd.DataFrame(columns=data.columns)
    
    for i in range(0,data.shape[0]):
        promt = top_3(data,i, n_similar)
        promt_df = pd.concat([promt_df, promt])
    return promt_df

In [66]:
res = new_df(df,3)

In [67]:
res

,record_name,ref_name,similarity_score
325,Гайка оцинкованная М16,"Гайки стальные оцинкованные шестигранные, диам...",52.173913
320,Гайка оцинкованная М14,"Гайки стальные оцинкованные шестигранные, диам...",52.173913
1973,Сетка сварная оцинкованная 50x50x4 мм,Сетка стальная сварная из оцинкованной проволо...,52.380952
111,Анкерный болт с гайкой 20*200,Болты анкерные с гайкой стальные фрикционные р...,50.000000
110,Анкерный болт КТ 16х150,Болты анкерные с гайкой стальные фрикционные р...,51.724138
...,...,...,...
2403,Эмаль Кремнийорганическая Термостойкая (8) 81....,Эмаль кремнийорганическая термостойкая КО-811К...,58.490566
2404,Кремнийорганическая эмаль КО-8104,Эмаль кремнийорганическая термостойкая КО-88,77.647059
873,Кабель ААШв-10 3х120 Камкабель,Кабель силовой гибкий с медными жилами КГЭ 3х1...,44.000000
1882,"Кабель ПВС 3*1,5 380V (бухта)",Провод силовой гибкий с медными жилами ПВС 3х4,44.897959


In [ ]:
def top_3(data, i, n_similar):
    data['similarity_score'] = calculate_skill_similarity(data['record_name'], data['record_name'][i])
    data = data.sort_values(by = 'similarity_score')
    return data.iloc[-(n_similar+1):-1]

In [ ]:
def calculate_similarity(ksr, pred):
    similarity = []

    for ksr_item in ksr:
        similarity += [fuzz.ratio(ksr_item, pred)]
    return similarity

In [70]:
def Similar_df(data, n_similar):
    sm_df = pd.DataFrame(columns=data.columns)
    top = pd.DataFrame(columns=data.columns)
    for i in range(0,data.shape[0]):
        
        similarity = []
        for ksr_item in ksr:
        similarity += [fuzz.ratio(ksr_item, pred)]
        
        data['similarity_score'] = calculate_skill_similarity(data['record_name'], data['record_name'][i])
        top = data.sort_values(by = 'similarity_score')
        top = top.iloc[-(n_similar+1):-1]
        
        sm_df = pd.concat([sm_df, top])
    return sm_df

In [71]:
res = Similar_df(df,3)

In [72]:
res

,record_name,ref_name,similarity_score
325,Гайка оцинкованная М16,"Гайки стальные оцинкованные шестигранные, диам...",52.173913
320,Гайка оцинкованная М14,"Гайки стальные оцинкованные шестигранные, диам...",52.173913
1973,Сетка сварная оцинкованная 50x50x4 мм,Сетка стальная сварная из оцинкованной проволо...,52.380952
111,Анкерный болт с гайкой 20*200,Болты анкерные с гайкой стальные фрикционные р...,50.000000
110,Анкерный болт КТ 16х150,Болты анкерные с гайкой стальные фрикционные р...,51.724138
...,...,...,...
2403,Эмаль Кремнийорганическая Термостойкая (8) 81....,Эмаль кремнийорганическая термостойкая КО-811К...,58.490566
2404,Кремнийорганическая эмаль КО-8104,Эмаль кремнийорганическая термостойкая КО-88,77.647059
873,Кабель ААШв-10 3х120 Камкабель,Кабель силовой гибкий с медными жилами КГЭ 3х1...,44.000000
1882,"Кабель ПВС 3*1,5 380V (бухта)",Провод силовой гибкий с медными жилами ПВС 3х4,44.897959
